In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [1]:
# with open("../data/lidar.pcd", "rb") as f:
#     byte = f.read(1)
#     while byte != b"":
#         # Do stuff with byte.
#         byte = f.read(1)
#         print(byte)



Data from the lidar sensor:
* min_distance: 0.5
* max_distance: 100
* rays: 32
* rotations: 10    
* measurements: 360    
* fov: 41.3300018310547
* angle: 10
* compensated: True 

### Load the point cloud

In [2]:
pcd = o3d.io.read_point_cloud("../data/lidar.pcd")
print(pcd)
arr = np.asarray(pcd.points)
print(arr)
# o3d.visualization.draw_geometries([pcd])

PointCloud with 11145 points.
[[-1.97735310e+00  5.76935463e-06 -1.17242849e+00]
 [-1.97728515e+00  3.45212743e-02 -1.17256761e+00]
 [-1.97640276e+00  6.90236911e-02 -1.17257905e+00]
 ...
 [-3.62439728e+01 -3.17092109e+00  6.00578737e+00]
 [-3.59373322e+01 -2.51295590e+00  5.94680309e+00]
 [-3.52411079e+01 -1.23063064e+00  5.82093382e+00]]


In [26]:
pc = o3d.geometry.PointCloud()
pc.points = o3d.utility.Vector3dVector(arr[:1*369])
print(pc.points)

std::vector<Eigen::Vector3d> with 361 elements.
Use numpy.asarray() to access data.


### Visualizing

In [27]:
o3d.visualization.draw_geometries([pc])

In [5]:
# Something about normals? 
# See http://www.open3d.org/docs/0.7.0/tutorial/Basic/pointcloud.html#vertex-normal-estimation
pc.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=10))
o3d.visualization.draw_geometries([pc], point_show_normal=True)

### Only keep points that are within a certain distance
Seems like the distance is not calculated in meters...

In [6]:
mdis = pcd.compute_mahalanobis_distance() # Mahalanobis distance to each point from center
approvedPoints = np.array([])
for point, dis in zip(arr, mdis):
    if (dis > 2 and dis < 10) or dis < 1:
        if len(approvedPoints) < 1:
            approvedPoints = np.array([point])
        else:
            approvedPoints = np.vstack([approvedPoints, point])
print(len(approvedPoints))

8951


In [7]:
np.average(mdis[:360])

0.4576711708063753

In [8]:
# Visualization of new point cloud
newCloud = o3d.geometry.PointCloud()
newCloud.points = o3d.utility.Vector3dVector(approvedPoints)
o3d.visualization.draw_geometries([newCloud])

## Annet

In [3]:
dv = pcd.compute_nearest_neighbor_distance()
np.average(dv)

0.4180552444391524

In [4]:
print("Downsample the point cloud with a voxel of 0.05")
downpcd = pcd.voxel_down_sample(voxel_size=0.01)
o3d.visualization.draw_geometries([downpcd])


Downsample the point cloud with a voxel of 0.05


In [5]:

print("Recompute the normal of the downsampled point cloud")
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([downpcd])


Recompute the normal of the downsampled point cloud


compute_mahalanobis_distance

crop(...)
 |      crop(*args, **kwargs)
 |      Overloaded function.
 |      
 |      
 |      1. crop(self, bounding_box)
 |          Function to crop input pointcloud into output pointcloud
 |      
 |      Args:
 |          bounding_box (open3d.geometry.AxisAlignedBoundingBox): AxisAlignedBoundingBox to crop points
 |      
 |      Returns:
 |          open3d.geometry.PointCloud
 |      
 |      2. crop(self, bounding_box)
 |          Function to crop input pointcloud into output pointcloud
 |      
 |      Args:
 |          bounding_box (open3d.geometry.OrientedBoundingBox): AxisAlignedBoundingBox to crop points
 |      
 |      Returns:
 |          open3d.geometry.PointCloud

In [32]:
# help(downpcd)
mdis = pcd.compute_mahalanobis_distance()
print(np.asarray(pcd.points)[0])
print((mdis[0:20]))


[-1.97735310e+00  5.76935463e-06 -1.17242849e+00]
DoubleVector[0.133173, 0.133377, 0.133546, 0.13378, 0.133993, 0.134278, 0.134648, 0.135216, 0.135857, 0.136596, 0.137606, 0.138855, 0.140122, 0.141674, 0.143354, 0.145004, 0.146977, 0.538194, 0.538089, 0.53798]


In [35]:

print("Print a normal vector of the 0th point")
print(downpcd.normals[0])
print("Print the normal vectors of the first 10 points")
print(np.asarray(downpcd.normals)[:10, :])
print("")


Print a normal vector of the 0th point
[0. 0. 1.]
Print the normal vectors of the first 10 points
[[ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 8.22327732e-01  8.39817724e-02 -5.62782518e-01]
 [-1.29269318e-03  4.20271100e-04 -9.99999076e-01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [-7.19252606e-02 -1.67461594e-06 -9.97410024e-01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [-3.36616127e-04 -1.33513029e-02 -9.99910811e-01]
 [-5.83226912e-03  1.02075167e-02 -9.99930893e-01]]



In [47]:
# plt.imshow(np.asarray(downpcd.RGBDImage))
downpcd.compute_mean_and_covariance()

(array([-1.37772025, -0.46522382, -0.80105595]),
 array([[442.5370421 , -20.33351605,  -8.24120481],
        [-20.33351605, 442.45936844,  -3.82447938],
        [ -8.24120481,  -3.82447938,   8.87620334]]))

In [133]:
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([downpcd], point_show_normal=True)

In [33]:
downpcd[:300]

TypeError: 'open3d.cpu.pybind.geometry.PointCloud' object is not subscriptable